In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from splits import split_users # contains split_users func

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 100)

In [2]:
# read in pickled DFs generated by query_dfs.py

df_orders = pd.read_pickle("./pickle/df_orders.pickle")
df_train = pd.read_pickle("./pickle/df_train.pickle")
df_prior = pd.read_pickle("./pickle/df_prior.pickle")
df_prod_detail = pd.read_pickle("./pickle/df_prod_detail.pickle")

In [3]:
# read in pickled feature DF generated by feature_engineering_1.ipynb
X = pd.read_pickle("./pickle/X_25.pickle")

In [4]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

0.24.1


In [5]:
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

X_train, X_test, y_train, y_test = split_users(X, seed=39, test_size=0.9)# only using 10%

Actual Test Size: 0.9003


In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=4000, max_depth=8, max_features=8, n_jobs=-1)
rf_fit = rf.fit(X_train, y_train)


In [ ]:
rf_fit.f1_score(X_test, y_test)

In [ ]:
with open(f"models/rf_fit.pickle", "wb") as pfile:
        pickle.dump(rf_fit, pfile)

In [ ]:
# X_train, X_test, y_train, y_test = split_users(X, seed=39, test_size=0.9) # only using 10%

from numpy import loadtxt
from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimators=4000, eval_metric="error")
xgb_fit = xgb.fit(X_train, y_train)

In [ ]:
with open(f"models/xgboost_fit.pickle", "wb") as pfile:
        pickle.dump(xgb_fit, pfile)

In [ ]:
xgb_fit.score(X_test, y_test)